This notebook is aimed at looking at some of the features that were engineered to run through the model, namely the columns containin the number of nearby observations and fires across time and space. Specifically, columns were created that hold the counts of observations and fires (denoted by the column `fire_bool == True`) that are within 0.1 km of a given observation, and within 1-7 days, as well as within 365, 730, 1095 days. These columns all have the base name `all_nearby_count` and `all_nearby_fires`, and then add on the number of days back in time that were used to look for nearby observations. For example, `all_nearby_count1` holds the number of observations that were within 0.1 km of a given observation, up to 1 day prior, whereas `all_nearby_fires2` holds the number of positively labeled observations (e.g. `fire_bool == True`) that were within 0.1 im of a given observation, up to 2 days prior. 

This notebook will be used to examine the distributions of these engineered columns. 